
# Pos Tagging-NER-Dependency Parsing
INCLUDE:

- [0. Basic Concepts](#1-basic-concepts)
    * [1. 隐马尔可夫模型（HMM）](#2-隐马尔可夫模型hmm)
    * [2. 条件随机场（CRF）](#3-条件随机场crf)
    * [3. POS Tagging：词性标注](#4-pos-tagging)
    * [4. Dependency Parsing：依存句法分析](#5-dependency-parsing)
    * [5. NER: Named Entity Recognition：命名实体识别](#5-ner-named-entity-recognition)
    * [6. Relation Extraction：实体关系提取](#6-relation-extraction)
- [1. Data Preprocessing (for POS Tagging--HMM)](#6-data-preprocessing-for-pos-tagging--hmm)
    * [1.1 Getting the Data](#7-getting-the-data)
    * [1.2 Splitting the Data into Train and Test Sets](#8-splitting-the-data-into-train-and-test-sets)
    * [1.3 Encode the Tokens(单词编码)](#9-encode-the-tokens)
    * [1.4 Encode the Tags(词性编码)](#10-encode-the-tags)
- [2. Hidden Markov Model (HMM) for POS Tagging](#11-hidden-markov-model--hmm--for-pos-tagging)
    - Transition Matrix & Start State Probabilities
    - Observation Matrix
    - Viterbi Algorithm: Predicting the Most Likely Sequence，即预测最可能的词性标记序列POS Tagging
    - Evaluation
    - NLTK库自带的HMM模型
- [3. Conditional Random Fields (CRF) for NER](#12-conditional-random-fields--crf--for-ner)
    - 数据准备：使用包含NER标注的数据集
    - NLTK库实现CRF模型：`CRFTagger`
    - 自定义实体提取器和评估函数：`extract_spans`和`cal_span_level_f1`
    - 优化CRFTagger: 自定义CRF类并添加`_get_features`来获取更多特征
    - 进一步优化：创建一个CRF类的子类，添加POS作为新特征
- [4. Dependency Parsing via spacy](#13-dependency-parsing-via-spacy)
    - 依存句法分析
    - 使用spacy库进行依存句法分析

上述都属于数据预处理内容，一般顺序为：Tokenization -> POS Tagging -> NER -> Dependency Parsing

In [1]:
import nltk
from nltk.corpus import brown
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import accuracy_score
from datasets import load_dataset

## 0. Basic Concepts
#### 判别模型 Discriminative Model
判别模型指的是直接对条件概率分布进行建模，而不是对联合概率分布进行建模。CRF是一种判别模型，它直接对输出序列的条件概率分布进行建模，而不是对输入和输出序列的联合概率分布进行建模。CRF的目标是学习一个条件概率分布$\(P(Y|X)\)$，其中$\(X\)$是输入序列，$\(Y\)$是输出序列。

#### 生成模型 Generative Model
与判别模型相对的是生成模型，它对输入和输出序列的联合概率分布进行建模。HMM是一种生成模型，它对输入和输出序列的联合概率分布进行建模。HMM的目标是学习一个联合概率分布$\(P(X, Y)\)$，其中$\(X\)$是输入序列，$\(Y\)$是输出序列。




## 1. 隐马尔可夫模型（HMM）

在本实验室中，我们将研究隐马尔可夫模型（HMM），以建立序列数据模型。HMM 基于马尔可夫假设，即现在的状态 $z_n$ 足以预测未来的状态 $y_{n+1}$，因此过去的状态 $y_{0:n-1}$ 可以被遗忘。

通常，我们感兴趣的状态是无法直接观察到的--它们是 "隐藏 "的。（POS）标记就是我们想要预测的隐藏状态。我们只能观察单词，并利用它们来推断标签。

马尔可夫模型基于马尔可夫链的数学理论。在时间序列中，假设有一个状态空间\(S = \{s_1, s_2, ..., s_N\}\)，其中每个\(s_i\)代表一个可能的状态。马尔可夫模型的核心是状态转移概率，即从一个状态转移到另一个状态的概率。

状态转移概率表示为：
$$\[P(s_{t+1} = j | s_t = i)\]$$

这表示在时间$\(t\)$处于状态$\(i\)$的条件下，在时间$\(t+1\)$转移到状态$\(j\)$的概率。

整个模型可以通过一个状态转移矩阵$\(A\)$来描述，其中矩阵的元素$\(a_{ij}\)$表示从状态$\(i\)$转移到状态$\(j\)$的概率：
$$\[A = [a_{ij}] = P(s_{t+1} = j | s_t = i)\]$$


#### 数学表达
隐马尔可夫模型包括三个主要的参数集：状态转移概率矩阵$\(A\)$，观测概率矩阵$\(B\)$，和初始状态概率向量$\(\pi\)$。

1. **状态转移概率矩阵**$\(A\)$:$\[A = [a_{ij}]\]$
其中，$\(a_{ij} = P(q_{t+1} = j | q_t = i)\)$，表示在时间$\(t\)$处于状态$\(i\)$的条件下，在时间$\(t+1\)$转移到状态$\(j\)$的概率。

2. **观测概率矩阵**$\(B\)$:$\[B = [b_j(k)]\]$
其中，$\(b_j(k) = P(o_t = v_k | q_t = j)\)$，表示在时间$\(t\)$处于状态$\(j\)$的条件下，观测到$\(v_k\)$的概率。$\(o_t\)$是在时间$\(t\)$的观测值，$\(v_k\)$是所有可能观测值的集合。

3. **初始状态概率向量**$\(\pi\)$: $\[\pi = [\pi_i]\]$
其中，$\(\pi_i = P(q_1 = i)\)$，表示在时间$\(t=1\)$时，系统处于状态$\(i\)$的概率。

因此，一个HMM可以由参数$\(\lambda = (A, B, \pi)\)$完全定义。

#### HMM 和 马尔可夫模型的区别

- **马尔可夫模型**关注于状态的转移概率，核心思想是未来的状态只依赖于当前的状态，而与之前的历史状态无关。
  
- **隐马尔可夫模型（HMM**则更进一步，引入了隐藏状态和观测值之间的关系。在HMM中，我们不能直接观察到状态，而是通过与这些状态相关联的观测值来推断状态的序列。这种模型特别适用于序列数据的处理，例如语音识别、自然语言处理中的词性标注等，其中系统的内部状态不能直接观察到，但可以通过观察到的行为（如单词序列）来推断。



## 2. 条件随机场（CRF）
CRF是一种判别模型，用于标注或分割序列数据。CRF是一种无向图模型，它可以用于对序列数据进行建模，特别适用于标注或分割任务。CRF的核心思想是对给定的输入序列，预测输出序列的条件概率分布。

CRF的输入是一个输入序列$\(X = (x_1, x_2, ..., x_n)\)$，输出是一个输出序列$\(Y = (y_1, y_2, ..., y_n)\)$。CRF的目标是学习一个条件概率分布$\(P(Y|X)\)$，即给定输入序列$\(X\)$，预测输出序列$\(Y\)$的概率。


CRF的关键概念是通过一组特征函数来定义模型，这些特征函数可以捕捉数据中的不同模式和依赖关系，从而帮助模型了解不同的输入观测序列应该如何影响相应的标签序列。

在CRF中，特征函数通常是二元或实值的，用于量化特定的观测与标签配置之间的关系。例如，一个特征函数可能检查一个词的前缀和后缀是否与特定的标签相匹配。特征函数的权重决定了这些模式在最终概率计算中的重要性。

训练CRF模型涉及最大化条件对数似然，这通常通过迭代优化算法如梯度下降完成。优化的目标是找到特征函数权重的最佳设置，以便模型最准确地预测训练数据中观察到的标签序列。

在使用CRF进行预测时，对于一个新的观测序列，我们会计算所有可能标签序列的条件概率，并选择概率最高的序列作为预测结果。这通常通过动态规划算法实现，如前面提到的维特比算法。

## 3. POS Tagging 
POS标注（Part-of-Speech tagging）是自然语言处理（NLP）中的一个基本任务，它涉及到将文本中的每个单词与一个特定的词性（如名词、动词、形容词等）相关联。POS标注的目标是识别出句子中每个单词的词性，这对于理解句子结构和意义非常重要。在很多NLP应用中，比如句法分析、语义分析、机器翻译和信息检索等，POS标注都是一个重要的预处理步骤。

POS标注通常利用统计模型或机器学习算法来自动完成，这些模型会根据单词本身、上下文信息以及单词之间的关系来预测每个单词的词性。随着深度学习的发展，基于深度学习的模型（如循环神经网络RNN和长短期记忆网络LSTM）在POS标注任务中也表现出了优异的性能。

我们将使用HMM模型来完成POS标注任务。HMM模型是一种生成式模型，它可以用于对观察序列进行建模，即根据观察序列预测隐藏状态序列。在POS标注任务中，观察序列是单词序列，隐藏状态序列是词性标记序列。HMM模型的训练过程涉及到估计转移概率矩阵和观察概率矩阵，这两个矩阵分别表示了词性标记之间的转移概率和单词与词性标记之间的关联概率。训练好的HMM模型可以用于对新的文本进行词性标注。

---

条件随机场（CRF）和隐马尔可夫模型（HMM）都可以用来进行词性标注（POS Tagging）。

**隐马尔可夫模型（HMM）** 是早期在自然语言处理领域，特别是在词性标注任务中非常流行的生成模型。HMM假设每个词的标签只依赖于它前面的一个或几个标签（一阶或高阶马尔可夫链），并且每个词的出现只依赖于它的标签，而忽略了其他词的影响。HMM通常使用维特比算法来预测给定句子中单词的词性标签序列。

**条件随机场（CRF）** 是一种判别模型，后来在POS Tagging中变得非常流行，因为它能够考虑更复杂的特征并捕捉数据中的长距离依赖关系。与HMM不同，CRF不对观测序列（即单词序列）进行建模，而是直接建模标签序列条件于观测序列的概率。这种模型特别适合于词性标注，因为在一个句子中，一个词的标签可能不仅仅依赖于它相邻的词的标签，而且还依赖于整个句子的上下文信息。

总的来说，HMM和CRF都可以用于词性标注，但CRF由于其能够考虑更广泛的上下文信息和特征依赖，通常会在这项任务上提供更准确的结果。

## 4. Dependency Parsing
依存句法分析（Dependency Parsing）是自然语言处理（NLP）中的一个重要任务，它旨在分析文本中单词之间的依存关系。在这种分析中，句子被表示为一个依存树（或图），其中节点代表句子中的单词，而边则代表单词之间的依存关系，通常是语法关系。

在依存句法分析中，每个句子都有一个根（通常是动词或句子的主要谓语），其他单词通过各种类型的依存关系与这个根连接。这些依存关系可以是主谓关系、定中关系、动宾关系等，它们帮助揭示句子的内在结构和意义。

依存句法分析的目的是帮助机器理解句子中词语的语法功能以及它们之间的相互作用，这对于很多NLP任务来说都是非常重要的基础，如信息提取、问答系统、机器翻译和文本摘要等。依存句法分析的结果可以提供丰富的语法信息，帮助提高这些任务的性能。依存句法分析通常在自然语言处理（NLP）的**文本预处理**阶段执行，特别是在需要深入理解句子结构和语义关系的任务中。执行这个任务可以帮助后续的分析，如情感分析、实体识别、关系抽取等，提供结构化的语言信息。

## 5. NER: Named Entity Recognition
命名实体识别（Named Entity Recognition, NER）是自然语言处理（NLP）中的一项重要任务，其目的是从文本中识别出具有特定意义的实体，如人名、地点名、组织名、时间表达式、数值表达式等，并将这些实体分类到预定义的类别中。NER在信息抽取、问答系统、内容摘要、情感分析等多种应用场景中扮演着关键角色。

#### NER的实现方法

1. **基于规则的方法**：这种方法依赖于手工编写的规则来识别和分类实体。规则可能包括词性标签、实体边界标识符（如大写字母开头）、上下文线索（比如位于“在”和“之间”的词组可能是地点）等。基于规则的方法简单直观，但缺乏灵活性，维护成本高，且难以覆盖所有情况。

2. **基于统计的方法**：利用机器学习模型，如隐马尔可夫模型（HMM）、条件随机场（CRF）、支持向量机（SVM）等，根据标注数据自动学习识别命名实体的规则。这些方法能够考虑到词语的上下文信息，对不确定性和多样性有更好的处理能力。

3. **基于深度学习的方法**：近年来，深度学习模型，特别是循环神经网络（RNN）、长短期记忆网络（LSTM）、双向长短期记忆网络（BiLSTM）和Transformer架构（如BERT、GPT），已经成为NER任务的主流方法。这些模型能够自动从大量数据中学习复杂的特征表示，捕捉更深层次的语言规律和实体间的依赖关系，从而提高识别的准确性和鲁棒性。

#### 应用实例

- **信息抽取**：从新闻、报告等文档中自动抽取关键实体，如人物、组织、地点，用于构建知识库或提供快速检索。
- **问答系统**：理解用户问题中的关键实体，并基于这些实体从数据库或知识库中检索正确的答案。
- **舆情分析**：识别社交媒体或评论中提及的产品、品牌或公司等实体，用于分析公众情感倾向或品牌影响力。
- **自动摘要**：识别文本中的关键实体和事件，生成内容摘要或概述。


## 6. Relation Extraction
关系抽取（Relation Extraction）是自然语言处理（NLP）中的一个重要任务，旨在从文本中识别实体之间的语义关系。例如，给定一句话"苹果公司是由乔布斯创立的"，关系抽取的任务是识别出"苹果公司"和"乔布斯"之间存在"创立者"的关系。

关系抽取的方法大致可以分为三类：

1. **基于规则的方法**：通过预定义的语言规则来识别实体之间的关系。这种方法简单直观，但缺乏灵活性，难以覆盖语言的多样性。

2. **基于特征的机器学习方法**：通过手工设计特征，使用传统的机器学习算法（如支持向量机、随机森林等）进行关系抽取。这种方法比基于规则的方法灵活，但需要大量的人工劳动和领域知识。

3. **基于深度学习的方法**：使用诸如卷积神经网络（CNN）、循环神经网络（RNN）、Transformer等深度神经网络自动从数据中学习表示，进行关系抽取。深度学习方法减少了对手工特征的依赖，能够更好地捕捉文本中的复杂模式，是当前关系抽取研究的热点。

关系抽取在信息抽取、知识图谱构建、问答系统等领域有着广泛的应用。随着深度学习技术的发展，关系抽取的准确性和应用范围都在不断扩大。

### 1. Classifier based approach
基于分类器的方法，是一种常用的关系抽取方法。它通常包括两个主要步骤：

**列出所有命名实体对**：首先，在句子中识别并列出所有命名实体（比如人名、组织名等）的组合。
**特征向量**：为每对实体生成的特征向量，它包含了对于分类器来说有用的信息，比如词性标注、依存关系、实体周围的词汇等。
**对每一对实体应用分类器**：对于每一对实体，使用一个分类器来预测它们之间是否存在某种关系。这里的类别标签可以是“无关系”或者是某种具体的关系类型

这种基于分类器的方式通常是监督学习模型。在监督学习中，模型会从标记好的训练数据中学习，这些数据包含了输入特征向量以及每个实例的正确标签（在关系抽取的场景中是实体对之间的关系类型）。模型的目的是通过学习这些示例来推广到新的未标记数据，预测实体对之间的关系类型。

### 2. Deep Learning based approach
如果使用深度学习方法，可以使用深度神经网络来自动学习特征表示。这种方式可直接输入原始数据（包含标记好的关系标签）到神经网络中，神经网络会自动学习特征表示并预测实体对之间的关系。

如果你的任务是多分类问题（即每个示例只能归于一个类别），你可能会在这个模型的输出上应用`softmax`函数来获取每个类别的概率。如果是多标签分类问题（即每个示例可能同时属于多个类别），则可能会使用`sigmoid`函数来独立地计算每个类别的概率。在实际应用中，选择哪种函数取决于你的具体任务需求。

# 1. Data Preprocessing (for POS Tagging--HMM)
### 1.1 Getting the Data

首先需要对文本进行数值转换。之前的Bag-of-Words模型是将文本转换为词频向量，而在POS标注任务中，需要将文本转换为词性标记序列。我们会先使用词性标注函数处理所有文本，得到一个list，该list每个元素是一个句子list，句子list中每个元素是一个元组，元组的第一个元素是单词，第二个元素是词性标记。

然后分别对单词和词性进行数值转换：
- 单词：使用gensim库中的Dictionary类，将单词转换为整数编码。
- 词性：使用sklearn库中的LabelEncoder类，将词性标记转换为整数编码。

---

1. `nltk.download('brown')`：这行代码下载Brown语料库。Brown语料库是第一个大规模的英语电子语料库，包含了不同风格和领域的文本，常用于语言学研究和自然语言处理任务中。

2. `nltk.download('universal_tagset')`：这行代码下载通用词性标记集。词性标记集（tagset）是一套预定义的词性标记，用于POS标注。通用词性标记集（Universal Tagset）是一个简化的、跨语言的词性标记集，旨在促进不同语言间的词性标注研究和应用。

3. `nltk_data = list(brown.tagged_sents(tagset='universal'))`：这行代码加载了Brown语料库，并将其句子以及这些句子中单词的词性标注（使用通用词性标记集进行标注）转换为一个Python列表。`brown.tagged_sents(tagset='universal')`这个函数调用会返回一个包含已标注句子的迭代器，每个句子都是一个单词及其词性标记的列表。通过将其转换为列表，便于后续的处理和分析。

核心内容在于使用`list(brown.tagged_sents(tagset='universal'))`将Brown语料库中的句子和词性标注转换为一个Python列表。这个列表的每个元素都是一个句子，每个句子都是一个元组列表，每个元组包含一个单词（token）和一个词性标记（tag）。

In [3]:
nltk.download('brown')  # download Brown corpus
nltk.download('universal_tagset')   # download the POS tags data
nltk_data = list(brown.tagged_sents(tagset='universal'))

[nltk_data] Downloading package brown to
[nltk_data]     C:\Users\yhb\AppData\Roaming\nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package universal_tagset to
[nltk_data]     C:\Users\yhb\AppData\Roaming\nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!


### 1.2 Splitting the Data into Train and Test Sets

- 提取出每个句子中的单词（token）和词性标记（tag），分别存储在两个列表中。
- 对训练集和测试集都进行上述处理。


In [4]:
# Split the data into training and test sets
train_set, test_set = train_test_split(
    nltk_data,
    train_size=0.80,  # use 80% as the training data
    test_size=0.20,
    random_state=101
)
print(f'Number of training sentences: {len(train_set)}')
print(f'Number of test sentences: {len(test_set)}')

# 下面是train_set[0]的输出结果：
# 可以认为train_set每个元素是一个句子，每个句子是一个元组列表，每个元组包含一个单词（token）和一个词性标记（tag）。
# [('A', 'DET'),
#  ('Newfoundland', 'NOUN'),
#  ('sat', 'VERB'),
#  ('solemnly', 'ADV'),
#  ('beside', 'ADP'),
#  ('a', 'DET'),
#  ('doghouse', 'NOUN'),
#  ('half', 'PRT'),
#  ('his', 'DET'),
#  ('size', 'NOUN'),
#  ('.', '.')]
# # 
# token为单词，即train/tes_set中的每个元组的第一个元素；
# tag为词性标记，即train/tes_set中的每个元组的第二个元素。

# Separate the labels from the text
train_toks = []  # each item in the list is a list of tokens in a document
train_tags = []  # each item in the list is a list of corresponding tags
for tagged_sentence in train_set:
    sentence_toks = []
    sentence_tags = []
    for token, tag in tagged_sentence:
        sentence_toks.append(token)
        sentence_tags.append(tag)

    train_toks.append(sentence_toks)
    train_tags.append(sentence_tags)

test_toks = []
test_tags = []
for tagged_sentence in test_set:
    sentence_toks = []
    sentence_tags = []
    for token, tag in tagged_sentence:
        sentence_toks.append(token)
        sentence_tags.append(tag)
    test_toks.append(sentence_toks)
    test_tags.append(sentence_tags)

print(f'Number of training sentences in train_toks: {len(train_toks)}')
print(f'Number of test sentences in test_toks: {len(test_toks)}')

Number of training sentences: 45872
Number of test sentences: 11468
Number of training sentences in train_toks: 45872
Number of test sentences in test_toks: 11468


### 1.3 Encode the Tokens(单词)
使用`gensim`库中的`Dictionary`类，将训练集和测试集中的单词转换为整数编码。`Dictionary`类是一个映射，用于将单词映射到整数编码。`doc2idx`方法可以将文档转换为整数编码的列表。
- train_toks_encoded: list; 训练集中的每个句子的每个词用对应的整数编码（idx）表示，这个列表的每个元素都是一个列表，代表了一个句子。

```python
train_toks_encoded[0]=[1, 2, 8, 10, 4, 3, 5, 6, 7, 9, 0]
# 表述train set的第一个句子的每个词的整数编码
```

预词袋模型类似，这都是将文本转换为数学表示的方法。不同的是，预词袋模型主要关注于词汇的出现频率而忽略了词序，而`Dictionary`和词汇ID映射保留了词序信息。
总结来说，词袋模型主要关注于词汇的出现频率而忽略了词序，适合于主题识别等任务；而Gensim的Dictionary和词汇ID映射保留了词序信息，适合于需要文本结构信息的任务


1. **建立`Dictionary`对象**：通过`Dictionary(train_toks + test_toks)`这行代码，你创建了一个`Dictionary`对象，它基于提供的文本数据（在这个案例中是训练集和测试集的合并）。这个对象内部构建了一个词汇表，其中包含了所有不重复的词汇以及每个词汇对应的唯一ID。`Dictionary`对象是一种高效的方式来管理词汇和它们的ID映射，这对于文本处理和模型训练都非常有用。

2. **使用`doc2idx`方法编码句子**：`doc2idx`方法接受一个词汇列表（即一个句子中的所有词汇）作为输入，并返回一个整数列表。该列表每个元素都是一个列表，其中每个值都是整数，代表了一个单词的映射idx。因此，`train_toks_encoded = [dictionary.doc2idx(sent) for sent in train_toks]`这行代码将训练数据集中的每个句子转换成了由词汇ID组成的列表，同样的，`test_toks_encoded`也经过了相同的处理。

通过这种方式，你可以将文本数据转换成数值形式，这是训练机器学习模型的常见和必要步骤。这样处理后的数据，即用词汇ID表示的句子，可以直接用于各种自然语言处理模型，如词嵌入模型、循环神经网络（RNN）等，来进行进一步的分析和学习。这个过程不仅减少了数据的维度（通过去除重复的词汇），也为后续的处理步骤（如词嵌入的学习）提供了便利。

In [5]:
from gensim.corpora import Dictionary

# 建立一个Dictionary对象，用于将单词映射到整数编码
dictionary = Dictionary(train_toks + test_toks)

# 使用dictionary对象的doc2idx方法将训练集和测试集中的单词转换为整数编码
train_toks_encoded = [dictionary.doc2idx(sent) for sent in train_toks]
test_toks_encoded = [dictionary.doc2idx(sent) for sent in test_toks]

# 查看一个句子的编码结果
print(f'Original sentence: {train_toks[3]}')
print(f'Encoded sentence: {train_toks_encoded[3]}')

Original sentence: ['many', 'of', 'their', 'gifted', 'members', 'were', 'prominent', 'in', 'the', 'Vatican', 'as', 'physicians', ',', 'musicians', ',', 'bankers', '.']
Encoded sentence: [41, 28, 46, 40, 42, 47, 45, 23, 31, 37, 38, 44, 11, 43, 11, 39, 0]


### 1.4 Encode the Tags(词性)
使用sklearn库中的`LabelEncoder`类，将训练集和测试集中的词性标记转换为整数编码。
- train_tags_encoded: list; 训练集中的每个句子的每个词性标记用对应的整数编码（idx）表示，这个列表的每个元素都是一个array，代表了一个句子,array的每个值是句子中每个词性标记的整数编码。
```python
train_tags_encoded[0]=[ 5  6 10  3  2  5  6  9  5  6  0]
# 表述train set的第一个句子的每个词的词性标记的整数编码
``` 

In [10]:
from sklearn.preprocessing import LabelEncoder

# 建立一个LabelEncoder对象，用于将词性标记映射到整数编码
label_encoder = LabelEncoder()

# 使用label_encoder对象的fit_transform方法将训练集和测试集中的词性标记转换为整数编码
label_encoder.fit([tag for sentence in train_tags for tag in sentence]) # 用训练集中的词性标记来训练label_encoder
train_tags_encoded = [label_encoder.transform(sentence) for sentence in train_tags] # 将训练集中的词性标记转换为整数编码
test_tags_encoded = [label_encoder.transform(sentence) for sentence in test_tags] # 将测试集中的词性标记转换为整数编码

In [12]:
print(f"List of tags: {label_encoder.classes_}") # 。classes_属性是一个列表，包含了所有不重复的词性标记
print(f"Mappings from tags to IDs:")
for tag in label_encoder.classes_:
    print(f"{tag}: {label_encoder.transform([tag])[0]}") # 给出上述不重复的词性标记的整数编码，共有12个不同的词性标记[0--11]

List of tags: ['.' 'ADJ' 'ADP' 'ADV' 'CONJ' 'DET' 'NOUN' 'NUM' 'PRON' 'PRT' 'VERB' 'X']
Mappings from tags to IDs:
.: 0
ADJ: 1
ADP: 2
ADV: 3
CONJ: 4
DET: 5
NOUN: 6
NUM: 7
PRON: 8
PRT: 9
VERB: 10
X: 11


# 2. Hidden Markov Model (HMM) for POS Tagging
HMMs的两个主要组成部分是转移模型(transition matrix)和观察模型(observation matrix)。
由上述数据集进行词性标记结果显示，该数据集一共由12个不同的词性标记。

#### Transition Matrix: 12x12
转移模型估计$P(tag_{t+1}|tag_t)$，即给定当前标记的情况下下一个标记的概率。
具体实现中，我们计算训练集中每个标记对的频率，即由标记$tag_t$转移到标记$tag_{t+1}$的频率。然后将这些频率归一化(除以每个标记的总数)以得到概率。
transition矩阵的每行表示$t-1$时刻的标记，每列表示$t$时刻标记，矩阵中的每个元素表示从第一个标记转移到第二个标记的频率。
#### Observation Matrix: 12xV
对于离散特征，例如标记，观察模型估计$P(word|tag_t)$，即给定当前标记的情况下观察到一个词的概率。
具体实现中，我们计算训练集中每个标记对应的词的频率，即由标记$tag_t$生成词$word$的频率。然后将这些频率归一化(除以每个标记的总数)以得到概率。
observations矩阵每行表示一个标记，每列表示一个词，矩阵中的每个元素表示对应标记生成对应词的频率。

### 2.1 Transition Matrix & Start State Probabilities
#### Transition Matrix
- 计算转移矩阵，即t-1时刻的标记转移到t时刻标记的概率。先填充转移次数。
- 计算转移概率矩阵A，每行元素=转移次数矩阵的每行元素除以每行的总和。
结果是一个12x12的矩阵。（12是不同的词性标记的数量）
#### Start State Probabilities
- 计算起始状态概率向量π，即每个标记在序列开始时出现的概率。
- 计算每个标记在序列开始时出现的次数，然后除以总次数得到概率。

In [13]:
num_tags = len(label_encoder.classes_)  # number of unique tags
transitions = np.zeros((num_tags, num_tags))  # 转移矩阵，初始时用转移次数填充
start_states = np.zeros(num_tags) # 开始状态概率，首先用每个状态在序列开始时出现的次数填充

# 遍历训练集中的每个句子，计算转移矩阵和开始状态概率 （只关注tag）
for sentence_tags in train_tags_encoded:
    for i, tag in enumerate(sentence_tags):
        if i == 0:
            # 若当前标记是句子的第一个标记，则更新初始状态矩阵
            start_states[tag] += 1
            continue
        previous_tag = sentence_tags[i-1]  # 获取前一个标签
        # 转移矩阵每个元素值表示从previous_tag转移到tag的次数（之后会计算频率，作为概率矩阵）
        transitions[previous_tag, tag] += 1  # 更新转移次数
        
# 计算转移概率矩阵A
transition_probabilities = np.zeros_like(transitions) # np.zeros_like()返回一个与给定数组具有相同形状和类型的零数组
for i in range(num_tags):
    total = np.sum(transitions[i]) # 计算每一行的总和
    if total > 0:
        # 转移矩阵A 的每行元素=转移次数矩阵的每行元素除以每行的总和
        transition_probabilities[i] = transitions[i] / total 

# 计算起始状态概率向量π
start_state_probabilities = start_states / np.sum(start_states)

# 打印结果以验证
print("转移概率矩阵A:\n", transition_probabilities)
print("起始状态概率π:\n", start_state_probabilities)

转移概率矩阵A:
 [[1.73216366e-01 4.65903639e-02 1.04461654e-01 7.00705915e-02
  1.12247276e-01 1.10204921e-01 1.33931876e-01 2.00123364e-02
  7.43746145e-02 2.91549585e-02 1.23761223e-01 1.97381948e-03]
 [1.00785399e-01 5.64290523e-02 8.77851747e-02 9.63422844e-03
  3.78038746e-02 5.87927294e-03 6.53033136e-01 7.25559129e-03
  3.75495549e-03 1.95078166e-02 1.76228589e-02 5.08639390e-04]
 [9.75938952e-03 8.21847882e-02 2.06089236e-02 1.52533721e-02
  1.90342703e-03 4.55766951e-01 2.58355612e-01 3.02904457e-02
  6.95443023e-02 1.45179571e-02 4.13908860e-02 4.23945112e-04]
 [1.70394487e-01 1.37658051e-01 1.40735455e-01 9.66260063e-02
  1.70149187e-02 7.32555806e-02 3.25134358e-02 1.35361149e-02
  4.78112526e-02 2.88785318e-02 2.41464666e-01 1.11500123e-04]
 [2.02334886e-02 1.11333377e-01 7.23421001e-02 9.19525152e-02
  2.29553355e-04 1.51898734e-01 2.43949629e-01 1.89545484e-02
  6.74558930e-02 2.51852824e-02 1.95874598e-01 5.90280055e-04]
 [1.29902207e-02 2.40081235e-01 9.11145061e-03 1.782953

### 2.2 Observation Matrix
观测矩阵是一个12xV的矩阵，V是词汇表的大小。每行对应一个标记，每列对应一个单词。矩阵中的每个元素表示在t时刻为tag_i时，观察到word_j的次数。之后用频率归一化得到概率矩阵B。

In [14]:
V= len(dictionary.values())  # vocabulary
observations = np.zeros((num_tags, V))  # We will first fill this with counts of words given tags

for i, sentence_toks in enumerate(train_toks_encoded):
    # i=25047; sentence_toks=[178, 1687, 37610, 378, 37608, 37609, 47, 37611, 0]
    # 这表示train_toks_encoded第25047个句子中的单词对应的编码
    
    sentence_tags = train_tags_encoded[i] # 用同样索引i取出
    for j, tok in enumerate(sentence_toks):
        tag = sentence_tags[j]
        observations[tag, tok] += 1        
# observations是一个12*V的矩阵，其中每行对应于一个标记，每列对应于一个单词。相当于记录在t时刻为tag_i时，观察到word_j的次数。
    
# 计算观察概率矩阵B
observation_probabilities = np.zeros_like(observations)
for i in range(num_tags):
    total = np.sum(observations[i])
    if total > 0:
        observation_probabilities[i] = observations[i] / total

# 打印结果以验证
print("观察概率矩阵B:\n", observation_probabilities)

观察概率矩阵B:
 [[0.33475022 0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.00092081 0.         ... 0.         0.         0.        ]]


### 2.3 Viterbi Algorithm
Viterbi算法是一个动态规划算法，用于在HMM中找到最可能的隐藏状态序列。在POS标注任务中，Viterbi算法用于找到给定一个单词序列的最可能的词性标记序列。

Input：
- observed_seq: 观测序列，即输入的句子
- num_tags: 标记的数量
- start_probs: 起始状态概率
- transition_probs: 转移概率矩阵
- observation_probs: 观察概率矩阵

Output：
- state_seq: 最可能的状态序列，即预测的词性标记序列

注意，Viterbi算法返回的结果是最可能的状态序列，即预测的词性标记序列。需要进行转换，将整数编码转换为原始的词性标记。

In [15]:
def viterbi(observed_seq, num_tags, start_probs, transition_probs, observation_probs):
    eps = 1e-7

    num_obs = len(observed_seq)

    # Initialise the V and backpointers
    V = np.zeros((num_obs, num_tags))
    backpointer = np.zeros((num_obs, num_tags))

    # For the first data point in the sequence:
    V[0, :] = start_probs * observation_probs[:, observed_seq[0]]

    # Run Viterbi forward for t > 0
    for t in range(1, num_obs):

        for state in range(num_tags):
            # probabilities for all the sequences leading to this state at time t
            seq_prob = V[t-1, :] * transition_probs[:, state]

            # Choose the most likely sequence
            max_seq_prob = np.max(seq_prob)
            best_previous_state = np.argmax(seq_prob)

            # Calculate the probability of the most likely sequence leading to this state at time t, including the current observation.
            # Add eps to help with numerical issues.
            V[t, state] = (max_seq_prob + eps) * (observation_probs[state, observed_seq[t]] + eps)

            backpointer[t, state] = best_previous_state

    t = num_obs - 1

    # Initialise the sequence of predicted states
    state_seq = np.zeros(num_obs, dtype=int)

    # Get the most likely final state:
    state_seq[t] = np.argmax(V[t, :])

    # Backtrack until the first observation
    for t in range(len(observed_seq)-1, 0, -1):
        state_seq[t-1] = backpointer[t, state_seq[t]]

    return state_seq

In [16]:
# 使用Viterbi算法对测试集中的每个句子进行词性标记预测
predictions = []
for sentence in test_toks_encoded:
    predictions.append(viterbi(sentence, num_tags, start_state_probabilities, transition_probabilities, observation_probabilities))
    
# 将整数编码转换为原始的词性标记
predicted_tags = [label_encoder.inverse_transform(pred) for pred in predictions]

# 打印一些结果以验证
examples = [2, 334]
for eg in examples:
    print(f'Tokens:      {test_toks[eg]}')
    print(f'Gold tag:    {test_tags[eg]}')
    print(f'Predictions: {predicted_tags[eg]}','\n')
    
# 计算准确率
correct = 0
total = 0
for i in range(len(test_tags)):
    correct += np.sum(np.array(test_tags[i]) == np.array(predicted_tags[i]))
    total += len(test_tags[i])
accuracy = correct / total
print(f'Accuracy: {accuracy:.4f}')

# 或使用sklearn库中的accuracy_score函数计算准确率
# all_predictions = [tag for sentence in predictions for tag in sentence]
# all_targets = [tag for sentence in test_tags_encoded for tag in sentence]
# acc = accuracy_score(all_targets, all_predictions)
# print(f'Accuracy = {acc}')

Tokens:      ['``', 'My', 'God', ',', "I'm", 'shot', "''", '!', '!']
Gold tag:    ['.', 'DET', 'NOUN', '.', 'PRT', 'VERB', '.', '.', '.']
Predictions: ['.' 'DET' 'NOUN' '.' 'PRT' 'NOUN' '.' '.' '.'] 

Tokens:      ['She', 'thought', 'she', 'was', 'bigger', 'than', 'we', 'are', 'because', 'she', 'came', 'from', 'Torino', "''", '.']
Gold tag:    ['PRON', 'VERB', 'PRON', 'VERB', 'ADJ', 'ADP', 'PRON', 'VERB', 'ADP', 'PRON', 'VERB', 'ADP', 'NOUN', '.', '.']
Predictions: ['PRON' 'VERB' 'PRON' 'VERB' 'ADJ' 'ADP' 'PRON' 'VERB' 'ADP' 'PRON' 'VERB'
 'ADP' 'NOUN' '.' '.'] 

Accuracy: 0.9437


### 2.5 NLTK---HiddenMarkovModelTrainer
NLTK库中的HiddenMarkovModelTrainer类可以用于训练HMM模型。HiddenMarkovModelTrainer类的train_supervised方法可以用于训练HMM模型，train_supervised方法的输入是训练数据，输出是训练好的HMM模型。

只需要将格式正确的数据划分为训练集和测试集，然后使用train_supervised方法即可训练HMM模型。训练好的模型可以用于预测新的数据，并且可以使用accuracy方法计算模型的准确率。

对于提升模型准确率的问题，由于HiddenMarkovModelTrainer类在train_supervised方法中没有直接提供调整模型复杂度或学习行为的参数，提升模型性能的途径主要依赖于训练数据的质量、量和预处理步骤。以下是一些可能帮助提高准确率的策略：

In [17]:
from nltk.tag.hmm import HiddenMarkovModelTrainer

# 训练数据和测试数据应该是这样的格式：[[(word1, tag1), (word2, tag2), ...], ...]
# 其中每个内层列表代表一个句子，句子由单词和对应标签的元组组成

# 创建一个HMM模型训练器
trainer = HiddenMarkovModelTrainer()

# 训练模型, 用train_supervised方法
hmm_model = trainer.train_supervised(train_set)

# 输出预测结果
print(test_set[0],'\n')
print(hmm_model.tag(test_toks[0]))

# 使用测试集评估模型
accuracy = hmm_model.accuracy(test_set)  # 注意，这里的test_data也应该是同样的格式

print(f"HMM模型的准确率: {accuracy:.4f}")

[('However', 'ADV'), (',', '.'), ('the', 'DET'), ('wei', 'X'), ('books', 'NOUN'), ('were', 'VERB'), ('also', 'ADV'), ('destroyed', 'VERB'), ('in', 'ADP'), ('a', 'DET'), ('series', 'NOUN'), ('of', 'ADP'), ('Orthodox', 'ADJ'), ('Confucian', 'ADJ'), ('purges', 'NOUN'), ('which', 'DET'), ('culminated', 'VERB'), ('in', 'ADP'), ('a', 'DET'), ('final', 'ADJ'), ('proscription', 'NOUN'), ('in', 'ADP'), ('605', 'NUM'), ('.', '.')] 


E:\Programming\anaconda3\envs\mytorch\lib\site-packages\nltk\tag\hmm.py:336: RuntimeWarning: overflow encountered in cast
  O[i, k] = self._output_logprob(si, self._symbols[k])
E:\Programming\anaconda3\envs\mytorch\lib\site-packages\nltk\tag\hmm.py:364: RuntimeWarning: overflow encountered in cast
  O[i, k] = self._output_logprob(si, self._symbols[k])


[('However', 'ADV'), (',', '.'), ('the', 'DET'), ('wei', 'DET'), ('books', 'DET'), ('were', 'DET'), ('also', 'DET'), ('destroyed', 'DET'), ('in', 'DET'), ('a', 'DET'), ('series', 'DET'), ('of', 'DET'), ('Orthodox', 'DET'), ('Confucian', 'DET'), ('purges', 'DET'), ('which', 'DET'), ('culminated', 'DET'), ('in', 'DET'), ('a', 'DET'), ('final', 'DET'), ('proscription', 'DET'), ('in', 'DET'), ('605', 'DET'), ('.', 'DET')]
HMM模型的准确率: 0.7492


# 3. Conditional Random Fields (CRFs) for Named Entity Recognition (NER)
这里我们使用CRF模型来完成命名实体识别（NER）任务。
首先加载`conll2003`数据集，这是一个常用的NER数据集，包含了英文句子和对应的命名实体标记。然后我们将数据集划分为训练集和测试集，用于训练和评估CRF模型。


In [55]:
cache_dir = "./data_cache"

# The data is already divided into training and test sets.
# Load the training set:
train_dataset = load_dataset(
    "conll2003",
    split="train",
    cache_dir=cache_dir,
)
print(f"Training dataset with {len(train_dataset)} instances loaded")

# Load the test set:
test_dataset = load_dataset(
    "conll2003",
    split="test",
    cache_dir=cache_dir,
)
print(f"Test dataset with {len(test_dataset)} instances loaded")

Training dataset with 14041 instances loaded
Test dataset with 3453 instances loaded


### 3.1 Data Preprocessing
首先查看数据集的格式。可以看到每个元素都是一个字典，对应了文本的一句话，以及这句话中每个单词的词性标记（pos）,分块标记（chunk）和命名实体标记（ner）。
这个数据集输出看起来是某个自然语言处理（NLP）任务的一部分，其中包含了句子的分词（tokens）、词性标注（pos_tags）、分块标签（chunk_tags）以及命名实体识别标签（ner_tags）。每个标签对应一种特定的标注，通常这些标注是预先定义好的。下面是对这些标签的基本解释：

1. 词性标注（POS Tagging）
`pos_tags`列表中的每个数字代表一个词性（Part-of-Speech, POS）的标识符，对应于句子中每个词的词性。词性包括名词、动词、形容词等。不同的数字代表不同的词性。具体到您提供的数据，这些数字（如22, 42, 16等）需要对照特定的词性标注方案来解释，比如Universal POS tags或者Penn Treebank POS Tags。

2. 分块标签（Chunking）
`chunk_tags`列表表示句子中每个词属于的短语或“块”的类型。分块通常用于标识名词短语（NP）、动词短语（VP）等。这里的数字同样代表不同的短语类型。例如：
分块标签通常用于“浅层句法分析”，帮助识别句子中的基本结构，但不进入深层次的句法细节。

3. 命名实体识别标签（NER）
`ner_tags`列表包含的数字代表句子中每个词是否属于某个命名实体，以及它们的实体类型。命名实体识别（NER）是指识别文本中具有特定意义的实体，如人名、地点、组织等。例如：

本部分主要关注NER，我们只需要提取NER，并定义映射关系，将NER标签映射为整数编码。将使用下面的映射：

```
{'O': 0, 'B-PER': 1, 'I-PER': 2, 'B-ORG': 3, 'I-ORG': 4, 'B-LOC': 5, 'I-LOC': 6, 'B-MISC': 7, 'I-MISC': 8}
```

In [56]:
train_dataset[0]

{'id': '0',
 'tokens': ['EU',
  'rejects',
  'German',
  'call',
  'to',
  'boycott',
  'British',
  'lamb',
  '.'],
 'pos_tags': [22, 42, 16, 21, 35, 37, 16, 21, 7],
 'chunk_tags': [11, 21, 11, 12, 21, 22, 11, 12, 0],
 'ner_tags': [3, 0, 7, 0, 0, 0, 7, 0, 0]}

In [57]:
ner_tag_mapping = {0: 'O', 1:'B-PER', 2:'I-PER', 3:'B-ORG', 4:'I-ORG', 5:'B-LOC', 6:'I-LOC', 7:'B-MISC', 8:'I-MISC'}

# 先遍历train_dataset和test_dataset，提取出每个句子：s
# 然后对每个句子s，提取出s['ner_tags']
# 最后将s['ner_tags']中的每个标签，用ner_tag_mapping将整数编码映射为对应的命名实体标签
train_set = [list(zip(s['tokens'], [ner_tag_mapping[tok] for tok in s['ner_tags']])) for s in train_dataset][:-1]
test_set = [list(zip(s['tokens'], [ner_tag_mapping[tok] for tok in s['ner_tags']])) for s in test_dataset][:-1]

# test_tokens的每个元素是一个句子的单词，test_tags的每个元素是一个句子的命名实体标签
test_tokens = [s['tokens'] for s in test_dataset][:-1]
test_tags = [[ner_tag_mapping[tok] for tok in s['ner_tags']] for s in test_dataset][:-1]

In [58]:
train_set[0]
# 此时我们训练集的每个元素只包含了每个句子的单词和对应的命名实体，格式为[(word1, ner1), (word2, ner2), ...]。

[('EU', 'B-ORG'),
 ('rejects', 'O'),
 ('German', 'B-MISC'),
 ('call', 'O'),
 ('to', 'O'),
 ('boycott', 'O'),
 ('British', 'B-MISC'),
 ('lamb', 'O'),
 ('.', 'O')]

### 3.2 Create CRF Model
使用nltk库中的`CRFTagger`类，可以创建和训练CRF模型。CRFTagger类的train方法可以用于训练CRF模型，train方法的输入是训练数据，输出是训练好的CRF模型。

假设你已经有了格式化的训练数据 train_set
train_set 应该是一个列表，其中包含句子，每个句子又是一系列（单词，标签）对
例如: `train_set = [[("The", "DET"), ("cat", "NOUN"), ...], [...]]`

#### 建立CRF模型
可以直接用下面的方式建立模型，但是进行封装函数的好处是可以在函数内部进行一些预处理，比如数据清洗，数据预处理等 并可以重复使用
且若没有在train方法中指定保存路径，模型会保存在内存中，当程序结束时会丢失
```python
tagger=CRFTagger()
tagger.train(train_set)
```

#### 使用训练好的模型进行预测
```python
tagger.tag_sents([['This', 'is','about','the','United','States','of','America']])
# 输出结果是：[[('This', 'O'), ('is', 'O'), ('about', 'O'), ('the', 'O'), ('United', 'B-LOC'), ('States', 'I-LOC'), ('of', 'O'), ('America', 'B-LOC')]]
```

In [6]:
import nltk
from nltk.tag import CRFTagger

def train_CRF_NER_tagger(train_set):
    # 创建CRF标记器的实例
    tagger = CRFTagger()

    # 使用训练集训练标记器, 并保存模型
    tagger.train(train_set, './nlp_models/model.crf.tagger')
    
    return tagger  # 返回训练好的模型

tagger = train_CRF_NER_tagger(train_set)

In [13]:
predicted_tags = tagger.tag_sents(test_tokens)
print(predicted_tags[0])
print(test_set[0])

[('SOCCER', 'O'), ('-', 'O'), ('JAPAN', 'B-LOC'), ('GET', 'O'), ('LUCKY', 'O'), ('WIN', 'O'), (',', 'O'), ('CHINA', 'B-ORG'), ('IN', 'O'), ('SURPRISE', 'O'), ('DEFEAT', 'O'), ('.', 'O')]
[('SOCCER', 'O'), ('-', 'O'), ('JAPAN', 'B-LOC'), ('GET', 'O'), ('LUCKY', 'O'), ('WIN', 'O'), (',', 'O'), ('CHINA', 'B-PER'), ('IN', 'O'), ('SURPRISE', 'O'), ('DEFEAT', 'O'), ('.', 'O')]


### 3.3 Evaluate the Model
实体匹配 vs. 标记匹配

- 正确匹配的实体：这指的是模型正确识别并分类的整个实体。在NER中，一个实体可能由一个词或多个连续的词组成。因此，只有当实体的所有组成部分都被准确识别并且分类正确时，才认为是一个正确的实体匹配。例如，如果实体“New York”被完整地识别为一个地点名（LOC），这就是一个正确的实体匹配。

- 正确标记的标记（Token）：这指的是单个词（或标记）被正确标注的情况。在一般的序列标注任务中，如词性标注，通常评估的是每个词的标注是否准确。但在NER中，即使单个词被正确标注，如果它是一个实体的一部分，而这个实体没有被完整且正确地识别出来，那么这并不足以说明模型的性能好。

由于nltk库等并没有提供直接计算NER准确率的函数，我们需要自己编写一个函数来进行评估。注意，我们需要自定义一个提取实体的函数，然后使用这个函数来提取实体，最后计算模型的准确率。


### 提取实体：`extract_spans(tagged_sents)`

这个函数的目的是从标记过的句子集合中提取出所有命名实体的跨度（span）。每个实体跨度由三个部分组成：实体开始的标记（token）索引、实体结束的标记索引，以及实体所在句子的索引。这个函数返回一个字典，其中键是实体类型（例如`"LOC"`、`"PER"`等），值是该类型实体的跨度列表。

函数逻辑如下：

- 遍历每个句子及其标记，使用`B-`和`I-`前缀来识别实体的开始和内部标记。
- 当遇到`B-`标记时，标记实体的开始，并记录实体类型。
- 遇到`I-`标记时，更新实体的结束位置。
- 遇到`O`标记或句子结束时，如果之前有开始标记，则将当前记录的实体添加到字典中，并重置开始标记。
- 函数返回包含所有实体跨度的字典，每种实体类型一个键。

打印出的span结果是一个字典，其中键为实体类型（'PER'、'LOC'、'ORG'等），值为包含实体跨度信息的列表。每个跨度是一个三元组(start, end, sidx)，其中：

start：实体开始的标记（token）索引。
end：实体结束的标记索引+1（即Python中的范围结束是独占的）。
sidx：实体所在句子的索引，从0开始。



In [ ]:
def extract_spans(tagged_sents):
    """
    Extract a list of tagged spans for each named entity type, 
    where each span is represented by a tuple containing the 
    start token and end token indexes.
    
    returns: a dictionary containing a list of spans for each entity type.
    """
    spans = {}
        
    for sidx, sent in enumerate(tagged_sents):
        start = -1
        entity_type = None
        for i, (tok, lab) in enumerate(sent):
            if 'B-' in lab:
                start = i
                end = i + 1
                entity_type = lab[2:]
            elif 'I-' in lab:
                end = i + 1
            elif lab == 'O' and start >= 0:
                
                if entity_type not in spans:
                    spans[entity_type] = []
                
                spans[entity_type].append((start, end, sidx))
                start = -1      
        # Sometimes an I-token is the last token in the sentence, so we still have to add the span to the list
        if start >= 0:    
            if entity_type not in spans:
                spans[entity_type] = []
                
            spans[entity_type].append((start, end, sidx))
                
    return spans

In [17]:
tagged_sents = [
    # 第一个句子
    [("John", "B-PER"), ("lives", "O"), ("in", "O"), ("New", "B-LOC"), ("York", "I-LOC"), (".", "O")],
    # 第二个句子
    [("She", "O"), ("works", "O"), ("at", "O"), ("Google", "B-ORG"), (".", "O")]
]
# 假设extract_spans函数已经定义

spans = extract_spans(tagged_sents)
print(spans)

# 对于'PER'（人名）实体类型，有一个实体跨度从第0句子（索引为0）的索引0开始到索引1结束。
# 对于'LOC'（地点名）实体类型，有一个实体跨度从第0句子的索引3开始到索引5结束。
# 对于'ORG'（组织名）实体类型，有一个实体跨度从第1句子（索引为1）的索引3开始到索引4结束。

{'PER': [(0, 1, 0)], 'LOC': [(3, 5, 0)], 'ORG': [(3, 4, 1)]}


### 评估：`cal_span_level_f1(test_sents, test_sents_with_pred)`

这个函数计算和打印出在测试数据上的实体层面F1分数。它首先使用`extract_spans`函数从真实标签和预测标签中提取实体跨度，然后对每个实体类型计算精确度、召回率和F1分数。

计算逻辑如下：

- 对于每种实体类型，通过比较预测的实体跨度和真实的实体跨度来计算真正例（TP）、假正例（FP）和假负例（FN）。
- 使用TP、FP和FN的值，计算每个实体类型的精确度、召回率和F1分数。
- 打印每个实体类型的F1分数，并计算所有实体类型F1分数的宏平均（Macro-average）作为总体性能指标。

这两个函数结合使用，提供了NER任务中一个详细的性能评估方法，特别是强调了实体层面的评估，而不仅仅是标记层面的正确性。这种评估方法更加符合NER任务的实际应用需求，因为在NER中，完整准确地识别出整个实体比仅正确标记实体中的单个词更为重要。

In [14]:
def cal_span_level_f1(test_sents, test_sents_with_pred):
    # get a list of spans from the test set labels
    gold_spans = extract_spans(test_sents)

    # get a list of spans predicted by our tagger
    pred_spans = extract_spans(test_sents_with_pred)
    
    # compute the metrics for each class:
    f1_per_class = []
    
    ne_types = gold_spans.keys()  # get the list of named entity types (not the tags)
    
    for ne_type in ne_types:
        # compute the confusion matrix
        true_pos = 0
        false_pos = 0
        
        for span in pred_spans[ne_type]:
            if span in gold_spans[ne_type]:
                true_pos += 1
            else:
                false_pos += 1
                
        false_neg = 0
        for span in gold_spans[ne_type]:
            if span not in pred_spans[ne_type]:
                false_neg += 1
                
        if true_pos + false_pos == 0:
            precision = 0
        else:
            precision = true_pos / float(true_pos + false_pos)
            
        if true_pos + false_neg == 0:
            recall = 0
        else:
            recall = true_pos / float(true_pos + false_neg)
        
        if precision + recall == 0:
            f1 = 0
        else:
            f1 = 2 * precision * recall / (precision + recall)
            
        f1_per_class.append(f1)
        print(f'F1 score for class {ne_type} = {f1}')
        
    print(f'Macro-average f1 score = {np.mean(f1_per_class)}')

cal_span_level_f1(test_set, predicted_tags)

F1 score for class LOC = 0.7970501474926254
F1 score for class PER = 0.7671940587665484
F1 score for class MISC = 0.6956521739130435
F1 score for class ORG = 0.6521877994251037
Macro-average f1 score = 0.7280210448993303


### 3.4 CRF模型的优化
可以通过自定义封装的CRF模型训练函数，对训练数据进行预处理，以及调整CRF模型的参数，来提升模型的性能。

首先定义`CustomCRFTagger`类并且在其中重写了_get_features方法后，通过创建一个CustomCRFTagger实例并使用这个实例来训练模型，训练过程会自动使用你定义的_get_features方法来提取特征。

然后定义一个CRF模型，并使用`CustomCRFTagger`类的train方法来训练模型。训练好的模型可以用于预测新的数据，并且可以使用accuracy方法计算模型的准确率。

In [18]:
import re, unicodedata
import nltk

class CustomCRFTagger(nltk.tag.CRFTagger):
    _current_tokens = None

    def __init__(self, *args, **kwargs):
        super(CustomCRFTagger, self).__init__(*args, **kwargs)
        self._pattern = re.compile(r'\d')  # 正则表达式用于检测数字
    
    def _get_features(self, tokens, idx):
        """
        Extract features for the word at position idx in the sentence.
        """
        token = tokens[idx]

        feature_list = []

        if not token:
            return feature_list

        # Capitalization
        if token[0].isupper(): # 当前单词的首字母是否大写
            feature_list.append("CAPITALIZATION")

        # Number
        if re.search(self._pattern, token) is not None: # 当前单词是否包含数字
            feature_list.append("HAS_NUM")

        # Punctuation
        # 这段代码检查给定的单词（token）是否完全由标点符号组成。它使用unicodedata.category来获取每个字符的Unicode类别，并检查这些类别是否全部属于标点符号的类别（如连接符（Pc）、破折号（Pd）、开括号（Ps）、闭括号（Pe）、初始引号（Pi）、终结引号（Pf）和其他标点符号（Po））。
        # 如果一个单词完全由标点符号组成，该函数会将"PUNCTUATION"特征添加到特征列表（feature_list）中。这对于自然语言处理（NLP）任务中识别标点符号的重要性很有帮助，因为标点符号可能对句子的结构和语义有显著影响。
        punc_cat = {"Pc", "Pd", "Ps", "Pe", "Pi", "Pf", "Po"}
        if all(unicodedata.category(x) in punc_cat for x in token):
            feature_list.append("PUNCTUATION")

        # Suffix up to length 3
        # 这段代码将单词的后缀添加到特征列表中。它将单词的后缀添加为特征，以便CRF标记器可以使用这些特征来识别单词。
        if len(token) > 1:
            feature_list.append("SUF_" + token[-1:])
        if len(token) > 2:
            feature_list.append("SUF_" + token[-2:])
        if len(token) > 3:
            feature_list.append("SUF_" + token[-3:])

        # Current word
        feature_list.append("WORD_" + token)

        # Previous word
        if idx > 0:
            prev_token = tokens[idx - 1]
            feature_list.append("PREV_WORD_" + prev_token)
        else:
            feature_list.append("BOS")  # Beginning of sentence

        # Next word
        if idx < len(tokens) - 1:
            next_token = tokens[idx + 1]
            feature_list.append("NEXT_WORD_" + next_token)
        else:
            feature_list.append("EOS")  # End of sentence

        return feature_list

In [21]:
# Train a CRF NER tagger
def train_CustomCRF_NER_tagger(train_set):
    tagger = CustomCRFTagger()
    tagger.train(train_set, './nlp_models/model_more_features.crf.tagger')
    return tagger  # return the trained model

tagger = train_CustomCRF_NER_tagger(train_set)

In [22]:
predicted_tags = tagger.tag_sents(test_tokens)
cal_span_level_f1(test_set, predicted_tags)

F1 score for class LOC = 0.8218497827436375
F1 score for class PER = 0.8260602335586971
F1 score for class MISC = 0.7557603686635945
F1 score for class ORG = 0.7094880991196609
Macro-average f1 score = 0.7782896210213975


#### 添加POS特征
可以继承上述定义的CustomCRFTagger类，并在此基础上添加PoS标签作为特征。
这也是一般的特征工程方法，即在原有的特征基础上添加新的特征，以提高模型的性能。

In [23]:
# 继承之前定义的CustomCRFTagger类，继承原来的特征提取方法，并在此基础上添加PoS标签作为特征
class CRFTaggerWithPOS(CustomCRFTagger):
    _current_tokens = None
    
    def _get_features(self, tokens, index):
        # Get the basic features from the parent class
        basic_features = super()._get_features(tokens, index)
        
        # Add the PoS tag as an additional feature
        pos_tag = nltk.pos_tag([tokens[index]])[0][1]
        
        # Append the PoS tag as an additional feature
        basic_features.append("POS_" + pos_tag)

        return basic_features

# 4. Dependency Parsing
NLTK库没有用于依存句法分析的内置模型，这里使用spacy库来完成依存句法分析任务。SpaCy的设计就是为了简化自然语言处理流程，让开发者能够通过一条简单的命令行调用完成复杂的文本分析任务。这种设计极大地提高了开发效率，让开发者可以集中精力在解决实际的问题上，而不是处理底层的NLP细节。

SpaCy提供nlp()函数，可以用于对文本进行分析。nlp()函数的输入是文本，输出是一个Doc对象，其中包含了对文本的分析结果。Doc对象是一个包含了分析结果的容器，其中包含了分词、词性标注、命名实体识别、依存句法分析等信息。
- `doc=nlp(text)`: 对文本进行分析，返回一个Doc对象。
- `doc.sents`: 获取文本中的句子。
- `for token in doc`: 遍历文本中的每个单词，每个单词是一个Token对象.
    - `token.text`: 获取单词的原始文本。
    - `token.pos_`: 获取单词的词性标注。
    - `token.dep_`: 获取单词的依存关系。
    - `token.head.text`: 获取单词的依存头部（父节点）。
- `for ent in doc.ents`: 获取文本中的命名实体。
    - `ent.text`: 获取命名实体的文本。
    - `ent.label_`: 获取命名实体的ner标签。

In [25]:
import spacy.cli
# 下载spacy的英文模型。如果需要分析其他语言的文本，可以下载其他语言的模型
spacy.cli.download("en_core_web_sm")
# 加载英文模型
nlp = spacy.load("en_core_web_sm")

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [53]:
# nlp()函数
text='I was wondering if anyone out there could enlighten me on this car I saw the other day. God bless America AND u.s.a.'
doc=nlp(text)

# Tagging
for token in doc:
    print(token.text, 'POS:', token.pos_, 'Dep:', token.dep_, 'Head:', token.head.text)

I POS: PRON Dep: nsubj Head: wondering
was POS: AUX Dep: aux Head: wondering
wondering POS: VERB Dep: ROOT Head: wondering
if POS: SCONJ Dep: mark Head: enlighten
anyone POS: PRON Dep: nsubj Head: enlighten
out POS: ADV Dep: advmod Head: there
there POS: ADV Dep: advmod Head: anyone
could POS: AUX Dep: aux Head: enlighten
enlighten POS: VERB Dep: advcl Head: saw
me POS: PRON Dep: dobj Head: enlighten
on POS: ADP Dep: prep Head: enlighten
this POS: DET Dep: det Head: car
car POS: NOUN Dep: pobj Head: on
I POS: PRON Dep: nsubj Head: saw
saw POS: VERB Dep: ccomp Head: wondering
the POS: DET Dep: det Head: day
other POS: ADJ Dep: amod Head: day
day POS: NOUN Dep: dobj Head: saw
. POS: PUNCT Dep: punct Head: wondering
God POS: PROPN Dep: nsubj Head: bless
bless POS: VERB Dep: ROOT Head: bless
America POS: PROPN Dep: dobj Head: bless
AND POS: CCONJ Dep: cc Head: America
u.s.a POS: NOUN Dep: conj Head: America
. POS: PUNCT Dep: punct Head: bless


In [54]:
# NER
for ent in doc.ents:
    print(ent.text, ent.label_)

the other day DATE
God PERSON
America GPE
